# Creazione dizionario dei sinonimi

In [1]:
import warnings
warnings.filterwarnings('ignore')
import os
import pandas as pd
import json

In [2]:
datasets_base_path = "..\\..\\Dataset\\Clusters_CSV\\original\\"
dictionary_path = ".\\dictionary\\"

In [3]:
cluster_dict = {}
for filename in os.listdir(dictionary_path):
    file_path = os.path.join(dictionary_path, filename)
    cluster = os.path.splitext(filename)[0]
    file = open(file_path)
    cluster_dict[cluster] = json.load(file)
    file.close()

In [4]:
threshold = 0.0
inverted_index_cluster = {}
for cluster, inverted_index in cluster_dict.items():
    inverted_index_cluster[cluster] = []
    for main_token, tokens in inverted_index.items():
        inverted_index_cluster[cluster].append({main_token: [k for k,v in tokens.items() if v > threshold]})